In [2]:
import gso_compare as compare
import os
dir_workspace = os.path.join(os.getcwd(),'workspace_GSO_Comparison')
file_name_cs = os.path.join(dir_workspace,'RSU_Custom.gso')
file_name_gs = os.path.join(dir_workspace,'RSU_Full.gso')
file_config = os.path.join(dir_workspace,'config.yaml')
df_summary,dict_result = compare.compare_gso(file_config,file_name_cs,file_name_gs,dir_workspace,False)
df_summary

KeyError: '++4appuseroto541'

In [ ]:
dict_result['occurrence']

dict_result['beField']